In [ ]:
using CSV, Tables
using JuMP
using Gurobi

import Pkg; Pkg.add("JSON3")
using JSON3

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [4]:
const GRB_ENV = Gurobi.Env(output_flag=1);

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-08


## load in data

In [5]:
all_classes = open("data/all_classes_info.json", "r") do file 
    JSON3.read(file)
end

R = CSV.File("data/ratings.csv",header=0) |> Tables.matrix;
H = CSV.File("data/hours.csv",header=0) |> Tables.matrix;
U = CSV.File("data/units.csv",header=0) |> Tables.matrix;

vars = CSV.File("data/variables.csv",header=0) |> Tables.matrix;
n = vars[1];
S = vars[2];

In [13]:
all_classes[1]

JSON3.Object{Base.CodeUnits{UInt8, String}, SubArray{UInt64, 1, Vector{UInt64}, Tuple{UnitRange{Int64}}, true}} with 28 entries:
  :rating              => 5.4
  :gir_attribute       => "REST"
  :has_final           => false
  :description         => "Presents engineering problems in a computational set…
  :offered_fall        => false
  :offered_spring      => true
  :meets_with_subjects => ["1.001"]
  :instructors         => ["J. Williams"]
  :out_of_class_hours  => 7.99
  :total_units         => 12
  :related_subjects    => ["2.156", "1.205", "1.C51", "1.000", "1.631", "1.063"…
  :pdf_option          => false
  :in_class_hours      => 5.7
  :is_half_class       => false
  :level               => "U"
  :prerequisites       => "GIR:CAL1"
  :subject_id          => "1.00"
  :title               => "Engineering Computation and Data Science"
  :lab_units           => 2
  :design_units        => 0
  :public              => true
  :offered_summer      => false
  :lecture_units       => 3
  :

In [16]:
findfirst(x -> x[:subject_id] == "8.01", all_classes)

3856

In [59]:
chem_gir_ids = ["3.091", "5.111", "5.112"]
phys_1_gir_ids = ["8.01", "8.011", "8.012", "8.01L"]
phys_2_gir_ids = ["8.02", "8.021", "8.022"]
math_1_gir_ids = ["18.01", "18.01A"]
math_2_gir_ids = ["18.02", "18.02A", "18.022"]
bio_gir_ids = ["7.012", "7.013", "7.014", "7.015", "7.016"]
gir_ids = [chem_gir_ids, phys_1_gir_ids, phys_2_gir_ids, math_1_gir_ids, math_2_gir_ids, bio_gir_ids]
gir_subject_types = [[findfirst(x -> x[:subject_id] == gir_id, all_classes) for gir_id in gir_type] for gir_type in gir_ids]

6-element Vector{Vector{Int64}}:
 [2962, 3303, 3304]
 [3856, 3857, 3858, 3859]
 [3860, 3861, 3862]
 [1536, 1537]
 [1538, 1540, 1539]
 [3752, 3753, 3754, 3755, 3756]

## optimize

In [67]:
model = Model(() -> Gurobi.Optimizer(GRB_ENV))

α = 0.8

# @variable(model, X[1:n,1:s] >=0)
@variable(model, X[1:n, 1:S], Bin)
@objective(model, Min, sum(α*H[i]*X[i,s] - (1-α)*R[i]*X[i,s] for i=1:n,s=1:S));

##
# CONSTRAINTS
##

# full-time; at least 36 units per semester
@constraint(model, [s in 1:S], sum(U[i]*X[i, s] for i=1:n) >= 36);

# wellbeing: no more than 112 hours per week
@constraint(model, [s in 1:S], sum(H[i]*X[i, s] for i=1:n) <= 112);

# not repeatable: cannot take a class again
# for i in 1:n
#     for t in 1:(S - 1)
#         @constraint(model, sum(X[i, t:end]) <= 1)
#     end
# end
@constraint(model, [i in 1:n], sum(X[i, t] for t=1:S) <= 1)

# GIR constraints
@constraint(model, [gir_type in gir_subject_types], sum(X[gir_ind, s] for gir_ind in gir_type, s in 1:S) == 1)

# 2 CI-H



1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, [[2962, 3303, 3304], [3856, 3857, 3858, 3859], [3860, 3861, 3862], [1536, 1537], [1538, 1540, 1539], [3752, 3753, 3754, 3755, 3756]]
And data, a 6-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 X[2962,1] + X[3303,1] + X[3304,1] + X[2962,2] + X[3303,2] + X[3304,2] + X[2962,3] + X[3303,3] + X[3304,3] + X[2962,4] + X[3303,4] + X[3304,4] + X[2962,5] + X[3303,5] + X[3304,5] + X[2962,6] + X[3303,6] + X[3304,6] + X[2962,7] + X[3303,7] + X[3304,7] + X[2962,8] + X[3303,8] + X[3304,8] = 1
 X[3856,1] + X[3857,1] + X[3858,1] + X[3859,1] + X[3856,2] + X[3857,2] + X[3858,2] + X[3859,2] + X[3856,3] + X[3857,3] + X[3858,3] + X[3859,3] + X[3856,4] + X[3857,4] + X[38

In [68]:
optimize!(model)

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[x86] - Darwin 23.6.0 23G93)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4725 rows, 37624 columns and 108768 nonzeros
Model fingerprint: 0xbb5bec89
Variable types: 0 continuous, 37624 integer (37624 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [2e-02, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 196.6940000
Presolve removed 551 rows and 4344 columns
Presolve time: 0.39s
Presolved: 4174 rows, 33280 columns, 99824 nonzeros
Found heuristic solution: objective 128.5800000
Variable types: 0 continuous, 33280 integer (33280 binary)
Found heuristic solution: objective 120.1900000

Root relaxation: objective 6.148400e+01, 128 iterations, 0.03 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds     

In [69]:
taken_classes_sem = [[ind for (ind, val) in enumerate(sem) if val == 1] for sem in eachcol(value.(X))]

8-element Vector{Vector{Int64}}:
 [974, 1025, 1180, 2021, 2962, 3857]
 [294, 1539, 1914, 1946, 2885, 2919, 3017, 4347, 4557]
 [100, 978, 1324, 3984]
 [714, 816, 3755]
 [1350, 2600, 3861]
 [1536, 4141, 4160]
 [296, 725, 1666, 3337, 4511]
 [144, 1953, 4492]

## Interpreter

In [27]:
function interpret_course_nums(classes_id_per_sem)
    course_nums = [[all_classes[class_ind][:subject_id] for class_ind in sem] for sem in classes_id_per_sem]
    return course_nums
end

interpret_course_nums (generic function with 1 method)

In [70]:
interpret_course_nums(taken_classes_sem)

8-element Vector{Vector{String}}:
 ["15.323", "15.3941", "15.839", "20.S900", "3.091", "8.011"]
 ["10.960", "18.022", "2.991", "20.001", "24.93", "3.006", "3.903", "HST.533", "SP.257"]
 ["1.267", "15.335", "16.634", "9.271"]
 ["12.834", "14.399", "7.015"]
 ["16.84", "21M.622", "8.021"]
 ["18.01", "CMS.627", "CMS.827"]
 ["10.991", "12.900", "18.727", "5.561", "MS.101"]
 ["1.713", "20.053", "MAS.921"]